In [1]:
#pip install


In [2]:
import pandas as pd

import numpy as np
import openai

In [4]:
PATH='./fine_food_reviews_1k.csv'
df = pd.read_csv(PATH)
df.head()

,Unnamed: 0,ProductId,UserId,Score,Summary,Text,combined,n_tokens,embedding
0,0,B003XPF9BO,A3R7JR3FMEBXQB,5,where does one start...and stop... with a tre...,Wanted to save some to bring to my Chicago fam...,Title: where does one start...and stop... wit...,52,"[0.006995313800871372, -0.027321254834532738, ..."
1,297,B003VXHGPK,A21VWSCGW7UUAR,4,"Good, but not Wolfgang Puck good","Honestly, I have to admit that I expected a li...","Title: Good, but not Wolfgang Puck good; Conte...",178,"[-0.0031342837028205395, -0.009958481416106224..."
2,296,B008JKTTUA,A34XBAIFT02B60,1,Should advertise coconut as an ingredient more...,"First, these should be called Mac - Coconut ba...",Title: Should advertise coconut as an ingredie...,78,"[-0.01757248304784298, -8.266512304544449e-05,..."
3,295,B000LKTTTW,A14MQ40CCU8B13,5,Best tomato soup,I have a hard time finding packaged food of an...,Title: Best tomato soup; Content: I have a har...,111,"[-0.0013932279543951154, -0.011112828738987446..."
4,294,B001D09KAM,A34XBAIFT02B60,1,Should advertise coconut as an ingredient more...,"First, these should be called Mac - Coconut ba...",Title: Should advertise coconut as an ingredie...,78,"[-0.01757248304784298, -8.266512304544449e-05,..."


In [5]:
#get the columns we need
df = df[['Score', 'Summary', 'Text', 'combined']]
df.head()

,Score,Summary,Text,combined
0,5,where does one start...and stop... with a tre...,Wanted to save some to bring to my Chicago fam...,Title: where does one start...and stop... wit...
1,4,"Good, but not Wolfgang Puck good","Honestly, I have to admit that I expected a li...","Title: Good, but not Wolfgang Puck good; Conte..."
2,1,Should advertise coconut as an ingredient more...,"First, these should be called Mac - Coconut ba...",Title: Should advertise coconut as an ingredie...
3,5,Best tomato soup,I have a hard time finding packaged food of an...,Title: Best tomato soup; Content: I have a har...
4,1,Should advertise coconut as an ingredient more...,"First, these should be called Mac - Coconut ba...",Title: Should advertise coconut as an ingredie...


In [6]:
#save df to csv
df.to_csv('fine_food_reviews.csv', index=False)

In [7]:
"""

Either use:

from sentence_transformers import SentenceTransformer
model = SentenceTransformer(MODEL_NAME)

or 
"""

#pip install InstructorEmbedding
#https://huggingface.co/hkunlp/instructor-large

from InstructorEmbedding import INSTRUCTOR
model = INSTRUCTOR('hkunlp/instructor-large')
MODEL_DIMENSIONS=768

#Check model embedding dimensions
sentence = "3D ActionSLAM: wearable person tracking in multi-floor environments"
instruction = "Represent the Science title:"
embeddings = model.encode([[instruction,sentence]])
len(embeddings)

/Users/watson/Library/Python/3.9/lib/python/site-packages/InstructorEmbedding/instructor.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange


load INSTRUCTOR_Transformer
max_seq_length  512


1

In [19]:
#function to calculate text embeddings from a text string
def create_embeddings(text):
    #text_embeddings = [ model.encode(sentence) for sentence in text]
    text_embeddings = model.encode(text)
    return text_embeddings

In [20]:
%%time
#apply the function to the combined column
df['instructor_embeddings'] = df.combined.apply(lambda x: create_embeddings(x))
print(df.head())

   Score                                            Summary  \
0      5  where does one  start...and stop... with a tre...   
1      4                   Good, but not Wolfgang Puck good   
2      1  Should advertise coconut as an ingredient more...   
3      5                                   Best tomato soup   
4      1  Should advertise coconut as an ingredient more...   

                                                Text  \
0  Wanted to save some to bring to my Chicago fam...   
1  Honestly, I have to admit that I expected a li...   
2  First, these should be called Mac - Coconut ba...   
3  I have a hard time finding packaged food of an...   
4  First, these should be called Mac - Coconut ba...   

                                            combined  \
0  Title: where does one  start...and stop... wit...   
1  Title: Good, but not Wolfgang Puck good; Conte...   
2  Title: Should advertise coconut as an ingredie...   
3  Title: Best tomato soup; Content: I have a har...   
4  T

In [21]:
#save embeddings to pickle file
df.to_pickle('./fine_food_reviews_with_instructor_embeddings_1k.pkl')
print('saved to pickle file')

saved to pickle file


In [ ]:
#or load df from pickle file

#PATH='./fine_food_reviews_with_intructor_embeddings_1k.pkl'
#df = pd.read_pickle(PATH)
#df = pd.read_csv(PATH)
#df.head()

In [22]:
# convert 5-star rating to binary sentiment
df = df[df.Score != 3]
df["sentiment"] = df.Score.replace({1: "Negative", 2: "Negative", 4: "Positive", 5: "Positive"})
df.head()

,Score,Summary,Text,combined,instructor_embeddings,sentiment
0,5,where does one start...and stop... with a tre...,Wanted to save some to bring to my Chicago fam...,Title: where does one start...and stop... wit...,"[-0.059960708, -0.009530962, 0.01014184, 0.034...",Positive
1,4,"Good, but not Wolfgang Puck good","Honestly, I have to admit that I expected a li...","Title: Good, but not Wolfgang Puck good; Conte...","[-0.055567216, 0.033855222, -0.0056050187, 0.0...",Positive
2,1,Should advertise coconut as an ingredient more...,"First, these should be called Mac - Coconut ba...",Title: Should advertise coconut as an ingredie...,"[-0.05114293, -0.00076456665, -0.002448596, 0....",Negative
3,5,Best tomato soup,I have a hard time finding packaged food of an...,Title: Best tomato soup; Content: I have a har...,"[-0.050088856, 0.021853218, -0.006671174, -0.0...",Positive
4,1,Should advertise coconut as an ingredient more...,"First, these should be called Mac - Coconut ba...",Title: Should advertise coconut as an ingredie...,"[-0.05114293, -0.00076456665, -0.002448596, 0....",Negative


In [23]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC

from sklearn.metrics import PrecisionRecallDisplay

In [24]:
#split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(list(df.instructor_embeddings.values), df['sentiment'], test_size=0.2, random_state=42)
y_test.value_counts()

sentiment
Positive    152
Negative     33
Name: count, dtype: int64

In [25]:
models = [
        ## n_estimators = number of trees in the forest
        ## random_state = seed used by the random number generator
    #RandomForestClassifier(n_estimators=200, max_depth=3, random_state=42),
    RandomForestClassifier(n_estimators=100, max_depth=3, random_state=42),
    LinearSVC(),
    #MultinomialNB(), #Needs non-negative values
    LogisticRegression(random_state=42, max_iter=10000),
]

In [26]:
for m in models:
    print(m)
    m.fit(X_train, y_train)
    y_pred = m.predict(X_test)
    #print(classification_report(y_test, y_pred))
    print(accuracy_score(y_test, y_pred))
    print('\n')

RandomForestClassifier(max_depth=3, random_state=42)
0.8648648648648649


LinearSVC()
0.9675675675675676


LogisticRegression(max_iter=10000, random_state=42)
0.8486486486486486




/Users/watson/Library/Python/3.9/lib/python/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


In [27]:
#Select best model
#LinearSVC() is the best model with 0.95 overall accuracy

linear_svc = LinearSVC()
linear_svc.fit(X_train, y_train)
y_pred = linear_svc.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

    Negative       1.00      0.82      0.90        33
    Positive       0.96      1.00      0.98       152

    accuracy                           0.97       185
   macro avg       0.98      0.91      0.94       185
weighted avg       0.97      0.97      0.97       185



/Users/watson/Library/Python/3.9/lib/python/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


In [28]:
#prompt user for input
#text = input("Enter text to embed: ")


#query='I did not like those noodles. They were too salty.'
query='I liked the ice cream, it was my favorite flavor'




In [29]:
#vectorize the query with the openai model
query_vector = np.array(create_embeddings(query)).astype(np.float32)

#vectorize with the instructor model
#query_vector = model_encoding.encode(query).astype(np.float32)


print(query_vector)

[-0.05388367  0.01293663 -0.00610974 -0.00064016  0.02779543 -0.00023739
  0.01041761  0.01520253 -0.03726192  0.06271198  0.04456315 -0.00146977
  0.02546944  0.0301823  -0.03854061 -0.01003428 -0.03777828  0.00566797
  0.00866211 -0.02147955  0.03131982  0.00214566 -0.00257729  0.03958992
  0.0268471   0.04376731 -0.03357205  0.00903402  0.0760744  -0.05456899
  0.02113983 -0.05553366 -0.01585522 -0.03614475 -0.01302151  0.01888763
  0.02554181  0.00808476 -0.00238992  0.02437423 -0.00769868  0.01059085
  0.00295327 -0.02198369  0.0030793  -0.03130994  0.00127087 -0.05831496
  0.02011083 -0.02193987 -0.04471128 -0.04107527 -0.05096359 -0.00053429
  0.0148061  -0.04691124 -0.06507159  0.0492857   0.05234139 -0.03496518
 -0.03952492  0.01231937 -0.0580942   0.00157165 -0.00154935 -0.06350773
  0.02888354 -0.05196198  0.01872565 -0.06729876  0.01732661 -0.03718399
 -0.01058599 -0.03071346 -0.03015275 -0.03773785  0.03404847  0.03995378
 -0.03403166  0.06799641  0.0441835   0.00477378  0

In [30]:
#predict on embedded text
y_predicted = linear_svc.predict([query_vector])

#y_predicted = linear_svc.predict([query_vector])

print()
print('Sentiment:  ' + y_predicted[0])
print()


Sentiment:  Positive

